<a href="https://colab.research.google.com/github/Pelosomelo/Misp/blob/main/roklad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp

Reading package lists... Done
Building dependency tree       
Reading state information... Done
coinor-cbc is already the newest version (2.9.9+repack1-1).
coinor-clp is already the newest version (1.16.11+repack1-1).
glpk-utils is already the newest version (4.65-1).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [ ]:
from pulp import *
import pandas as pd
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dane = pd.read_csv('/content/drive/MyDrive/rozklad.csv',index_col=0).to_dict()
pon = dane['PN']
Pracownicy=list(pon.keys())
print(dane)
Dni = ["PN","WT","SR","CZ","PT"]
max={'PN':2,'WT':1,'SR':1,'CZ':1,'PT':3}
#place={'Ania':{100},'Stefan':{50},'Hektor':{60},'Olaf':{40},'Lidia':{110},'Piotr':{70}}
#max=[2,1,1,1,3]
#place=[100,50,60,40,110,70]

wiersz=len(Pracownicy)
kolumna=len(Dni)

plan = [[0 for col in range(kolumna)] for row in range(wiersz)]
rozklad = [[0 for col in range(kolumna)] for row in range(wiersz)]



for w in range(wiersz):
    for k in range(kolumna):
        plan[w][k]=dane[Dni[k]][Pracownicy[w]]

plan = makeDict([Pracownicy,Dni],plan,0)

vars = LpVariable.dicts("Grafik",(Pracownicy,Dni),0,1,LpInteger)
prob = LpProblem("Grafik",LpMinimize)



for w in Pracownicy:
    for k in Dni:
        prob += vars[w][k]<=plan[w][k] 

for w in Pracownicy:
    prob += lpSum([vars[w][k] for k in Dni])<=3

for k in Dni:
    prob += lpSum([vars[w][k] for w in Pracownicy])==max[k]



prob.writeLP("Grafik_Pracownikow.lp")
prob.solve()
print("Status:", LpStatus[prob.status])

for v in range(wiersz*kolumna):
    rozklad[math.floor(v/kolumna)][v%kolumna] = prob.variables()[v].varValue

print(Dni)
for i in range(wiersz):
    print(Pracownicy[i],rozklad[i])

{'PN': {'Ania': 1, 'Stefan': 1, 'Hektor': 1, 'Olaf': 1, 'Lidia': 0, 'Piotr': 1}, 'WT': {'Ania': 1, 'Stefan': 0, 'Hektor': 1, 'Olaf': 1, 'Lidia': 0, 'Piotr': 1}, 'SR': {'Ania': 1, 'Stefan': 0, 'Hektor': 1, 'Olaf': 1, 'Lidia': 0, 'Piotr': 1}, 'CZ': {'Ania': 1, 'Stefan': 0, 'Hektor': 0, 'Olaf': 1, 'Lidia': 1, 'Piotr': 0}, 'PT': {'Ania': 1, 'Stefan': 0, 'Hektor': 0, 'Olaf': 1, 'Lidia': 1, 'Piotr': 0}}
Status: Optimal
['PN', 'WT', 'SR', 'CZ', 'PT']
Ania [0.0, 0.0, 1.0, 0.0, 0.0]
Stefan [0.0, 1.0, 0.0, 1.0, 1.0]
Hektor [1.0, 0.0, 1.0, 0.0, 0.0]
Olaf [0.0, 0.0, 1.0, 0.0, 0.0]
Lidia [0.0, 1.0, 0.0, 0.0, 0.0]
Piotr [0.0, 0.0, 0.0, 0.0, 0.0]
